In [1]:
import torch
from data_logger import DataLogger
from dqn_agent import Agent 
from utils import *
print(torch.cuda.is_available())

True


In [2]:
episodes = 200
frame_shape = (84, 84)

In [3]:
env = get_env(game="mario", level="SuperMarioBros-1-1-v0", action_space="SIMPLE_MOVEMENT")
agent = Agent(env, observation_size=(4,)+frame_shape, action_size=get_action_space_size(env), exploration_rate=1, discount_factor=0.995)#initial value for epsilon=1
env = apply_wrappers(env, skip=4, gray_scale=True, shape=frame_shape, num_stack=4)
logger = DataLogger(model="DDQN")

#count=0
for episode in range(episodes):
    print(f"---Episode {episode}")
    state = env.reset()
    time_stamp = 0
    prev_live = 3
    while time_stamp<1e4:
        action_idx = agent.act(state)

        next_state, reward, done, info = env.step(get_action(action_idx, env))

        agent.append_experience_to_memory(state, next_state, action_idx, reward, done)
        
        loss = agent.learn()    

        logger.add(loss, agent.step)

        state = next_state

        env.render()
        
        time_stamp += 1

        #lost_life = (info["lives"]!=prev_live)
        #prev_live = info["lives"]
        #count+=1
        #if count%100==0:
        #    print(f"Adding element {count} to memory")
        if done:# or lost_life:
            break
        

logger.close()

---Episode 0


KeyboardInterrupt: 

In [5]:
time_stamp=0
while time_stamp<1e4:
        action_idx = agent.act(state)

        next_state, reward, done, info = env.step(get_action(action_idx, env))

        state = next_state

        env.render()
        
        time_stamp += 1

        lost_life = (info["lives"]!=prev_live)
        prev_live = info["lives"]
        #count+=1
        #if count%100==0:
        #    print(f"Adding element {count} to memory")
        if done or lost_life:
            break

KeyboardInterrupt: 

: 

In [6]:
env.close()